In [1]:
award_names = ['Best Motion Picture – Drama', 'Best Motion Picture – Musical or Comedy', 'Best Motion Picture – Foreign Language',
 'Best Motion Picture – Animated', 'Best Director – Motion Picture', 'Best Actor in a Motion Picture – Drama',
 'Best Actor in a Motion Picture – Musical or Comedy', 'Best Actress in a Motion Picture – Drama',
 'Best Actress in a Motion Picture – Musical or Comedy', 'Best Supporting Actor – Motion Picture',
 'Best Supporting Actress – Motion Picture', 'Best Screenplay – Motion Picture', 'Best Original Score – Motion Picture',
 'Best Original Song – Motion Picture', 'Best Television Series – Drama', 'Best Television Series – Musical or Comedy',
 'Best Miniseries or Motion Picture – Television', 'Best Actor in a Television Series – Drama',
 'Best Actor in a Television Series – Musical or Comedy', 'Best Actor in a Miniseries or Motion Picture – Television',
 'Best Actress in a Television Series – Drama', 'Best Actress in a Television Series – Musical or Comedy',
 'Best Actress in a Miniseries or Motion Picture – Television',
 'Best Supporting Actor in a Series, Miniseries or Motion Picture – Television',
 'Best Supporting Actress in a Series, Miniseries or Motion Picture – Television']
new_award_names = [0]*len(award_names)
for i in range(len(award_names)):
    new_award_names[i] = award_names[i].lower()
#     new_award_names[i] = award_names[i].replace(" in ", " ").replace(" a ", " ").replace(" or ", " ").replace(" – ", " ").replace(" - ", " ").lower().split()
#     if "television" in new_award_names[i]:
#         new_award_names[i].append("tv")
new_award_names

['best motion picture – drama',
 'best motion picture – musical or comedy',
 'best motion picture – foreign language',
 'best motion picture – animated',
 'best director – motion picture',
 'best actor in a motion picture – drama',
 'best actor in a motion picture – musical or comedy',
 'best actress in a motion picture – drama',
 'best actress in a motion picture – musical or comedy',
 'best supporting actor – motion picture',
 'best supporting actress – motion picture',
 'best screenplay – motion picture',
 'best original score – motion picture',
 'best original song – motion picture',
 'best television series – drama',
 'best television series – musical or comedy',
 'best miniseries or motion picture – television',
 'best actor in a television series – drama',
 'best actor in a television series – musical or comedy',
 'best actor in a miniseries or motion picture – television',
 'best actress in a television series – drama',
 'best actress in a television series – musical or comedy'

In [2]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from collections import defaultdict

In [16]:
df = pd.read_json('../gg2015.json')
df = df['text']
df = df[~df.str.contains('think|should|maybe', case = False)]
df = df.str.replace('#GoldenGlobes|golden|globes|globe', "", case = False)
wins_df = df[df.str.contains('present|gave', case = False)]

wins_df

72         RT @hollywoodawards: Pointing at YOU #JLovers!...
78         RT @AmirArison: Tonight on @nbc rooting for #J...
180        RT @Latystarr: Rob deserved a GG nom for The R...
211        RT @GoldDerby: EXCLUSIVE Full rundown of tonig...
268        My  part 3:\nBest actor, drama: holy shit is t...
                                 ...                        
1753981    RT @RealsticLover: @JLo rocked the  !!! That's...
1754033    RT @peoplemag: That time everyone at the  frea...
1754061    RT @peoplemag: That time everyone at the  frea...
1754085    RT @JamieDornanUK: Jamie and Dakota presenting...
1754139    Rosamund Pike at The Weinstein Company &amp; N...
Name: text, Length: 28863, dtype: object

In [17]:
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load('en_core_web_md')
from string import punctuation

In [18]:
pattern1 = [{"ENT_TYPE": "PERSON", 'OP':'+'}, {"LEMMA": "present"}, {"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]
#pattern2 = [{"TEXT": "goes to"}, {"ENT_TYPE": "PERSON", 'OP':'+'}] might need to split goes to into two "text"
#another pattern: use work of art instead of "Person" to get movies etc too.
#can also try using work of art after "won" to see if that turns up anything else
# POS: very instead of lemma win

In [19]:
matcher = Matcher(nlp.vocab)
matcher.add("Winner", [pattern1])

In [20]:
winners = {}
winners = defaultdict(lambda: 1, winners)
for i, text in wins_df.iteritems():
    doc = nlp(text)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        winners[str(span).strip(punctuation).strip()] = winners[str(span).strip(punctuation).strip()] +1

In [21]:
winners = sorted(winners.items(), key=lambda item: item[1], reverse = True)
print(winners)

[('Jennifer Aniston presenting Best Supporting Actor', 4), ('Adam Levine present Best TV Series Drama', 4), ('Anna Ferris presenting Best Actress TV Series Drama', 4), ('Seth Meyers presenting Best Supporting Actor', 3), ('Jared Leto presenting Best Supporting Actress', 3), ('Kate Hudson presenting Best Film nominee #IntoTheWoods', 3), ('Jack Black presenting Best Film nominee', 3), ('Katherine Zeta-jones presents Best Film nominee #pride', 3), ('Harrison Ford presents Best Director award', 3), ('Jennifer Aniston presenting Best Supporting Actor  http://t.co/1XmzTEjVjk', 2), ('Benedict Cumberbatch present Best Supporting Actor', 2), ('Naomi Watts presenting Best Picture nominee #Birdman http://t.co/h53gY3MOCX', 2), ('Kerry Washington presented Best Actress TV Comedy', 2), ('Melissa McCarthy presenting Best Film nominee #SaintVincent', 2), ('Katie Holmes presented Best Supporting Actor', 2), ('Anna Faris presenting Best Actress TV series', 2), ('Oprah presenting Best Film nominee #Selma

In [22]:
import Levenshtein

award_list = [dict() for x in range(len(new_award_names))]

for tweet in winners: 
    tweet = tweet[0]
    split = tweet.lower().split()
    
    index = split.index("best")

    award_phrase = " ".join(split[index:])

    name_count = [0]*len(new_award_names)
    for i in range(len(new_award_names)):
#         match_length = len(set(new_award_names[i]).intersection(award_phrase))
#         try Levenshtein distance instead, may be more accurate
        
        name_count[i] = Levenshtein.distance(new_award_names[i], award_phrase)

    award_index = np.argmin(name_count)
    # WHAT ABOUT TIE? Neither? Or Both?
    #print(award_index)

#     if name_count[award_index] > 1:       
    if "wins" in split:
        person_index = split.index("wins")
    if "won" in split:
        person_index = split.index("won")
    person = split[:index-1]
    person = " ".join(person)
    if person not in award_list[award_index]:
        award_list[award_index][person] = 1
    else:
        award_list[award_index][person] = award_list[award_index][person]+1

print(award_list)

[{'kate hudson': 1, 'jack black': 1, 'katherine zeta-jones': 1, 'kerry washington': 1, 'oprah': 1, 'amy adams': 1, 'channing tatum': 2, 'gwyneth paltrow': 1, 'meryl streep': 2}, {}, {}, {'melissa mccarthy': 1}, {'jennifer aniston': 1, 'seth meyers': 1, 'harrison ford': 1, 'benedict cumberbatch': 1, 'katie holmes': 1}, {'naomi watts': 1}, {}, {}, {}, {'jennifer aniston': 1}, {'jared leto': 1}, {}, {}, {}, {'adam levine': 1, 'anna ferris': 1, 'anna faris': 1}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]


In [24]:
import operator
#AWARDS HAVE 1-2 PRESENTERS
i = 0
awards = {}
for award in award_list:
    if award != {}:
        awards[i] = max(award.items(), key=operator.itemgetter(1))[0]
    else:
        awards[i] = ''
    i+=1
print(awards)

{0: 'channing tatum', 1: '', 2: '', 3: 'melissa mccarthy', 4: 'jennifer aniston', 5: 'naomi watts', 6: '', 7: '', 8: '', 9: 'jennifer aniston', 10: 'jared leto', 11: '', 12: '', 13: '', 14: 'adam levine', 15: '', 16: '', 17: '', 18: '', 19: '', 20: '', 21: '', 22: '', 23: '', 24: ''}


In [38]:
tweet_phrase = ["supporting", "actor"]
award1phrase = ['director', '-', 'motion', 'picture']
award2phrase = ['supporting', 'actor', '-', 'motion', 'picture']

award_1diff = 0

for word in tweet_phrase: #whichever is shorter
    array = []
    for phrase in award1phrase:
        array.append(Levenshtein.distance(word, phrase))
    min_val = array[np.argmin(array)]
    award_1diff += min_val
    
print(award_1diff)

award_2diff = 0
for word in tweet_phrase: #whichever is shorter
    array = []
    for phrase in award2phrase:
        array.append(Levenshtein.distance(word, phrase))
    min_val = array[np.argmin(array)]
    award_2diff += min_val
    
print(award_2diff)

# print(Levenshtein.distance("supporting actor", 'director – motion picture'))
# print(Levenshtein.distance("supporting actor", 'supporting actor – motion picture'))


11
0
